## Convert Dictionaries into JSON file

In [ ]:
from LANoireTools.newscripts.dictionaries import FILE_FULLNAME_DICTIONARY
from json import dump

with open("name_hashes.json", "w") as file:
	dump({
		"file_name_hashes": FILE_FULLNAME_DICTIONARY
	})

In [ ]:
from zlib import decompress

IN_FILE = "X:\\SteamLibrary\\steamapps\\common\\L.A.Noire\\final\\pc\\cases.big.pc"
# IN_FILE = "..\\dump\\out\\files\\out\\trunk\\models\\la\\environment\\weather\\sun.trunk.pc"
OUT_FILE = "out2.unk"
OFFSET = 463264
SIZE = 32704 # - OFFSET
COMPRESSED = True

with open(IN_FILE, "rb") as file:
	file.seek(OFFSET)
	with open(OUT_FILE, "wb") as out:
		data: bytes = file.read(SIZE)
		if COMPRESSED:
			data = decompress(data, -15)
		out.write(data)

In [ ]:
PATH = "..\\dump\\vehicles\\files\\intermediate\\chunks\\vehicles\\"
FILES = ["ford_2dr_1947.chunk", "services\\ford_4dr_1947_police_vehicle.chunk", "chrysler_town_n_country_1946.chunk", "cadillac_ser_61_1947.chunk"]

RANGE = 144

out = [""] * (RANGE // 4)

for file in FILES:
	with open(PATH + file, "rb") as car:
		data: str = car.read(RANGE).hex()
		for i in range(RANGE // 4):
			out[i] += data[8 * i : 8 * (i + 1)] + "\t"

for s in out:
	print(s)

In [ ]:
from os import makedirs
from os.path import dirname, join
from struct import unpack
from json import dump

# FILE_NAME = "examples/cases.big.pc"
# FILE_NAME = "examples/audio.big.pc"
FILE_NAME = "examples/out.wad.pc"
OUT_FILE = "file_data.json"
OUT_DIRECTORY = "dump"

In [ ]:
from io import IOBase

class FileStream():
	def __init__(self, file: IOBase):
		self._file = file
		self._endian = "@"

	def set_endian(self, endian: str):
		self._endian = endian

	def tell(self):
		return self._file.tell()

	def seek(self, offset: int, end: int):
		self._file.seek(offset, end)

	def read(self, format: str, length: int):
		return unpack(f"{self._endian}{format}", self._file.read(length))[0]

	def read_uint8(self):
		return self.read("B", 1)
		
	def read_int8(self):
		return self.read("b", 1)
		
	def read_uint16(self):
		return self.read("H", 2)
		
	def read_int16(self):
		return self.read("h", 2)

	def read_uint32(self):
		return self.read("I", 4)
		
	def read_int32(self):
		return self.read("i", 4)
		
	def read_uint64(self):
		return self.read("Q", 8)
		
	def read_int64(self):
		return self.read("q", 8)

	def read_float32(self):
		return self.read("f", 4)
		
	def read_float64(self):
		return self.read("d", 8)

	def read_string(self, length: int):
		return self.read(f"{length}s", length)

	def read_pad(self, length: int):
		self.read_chunk(length)

	def read_chunk(self, length: int):
		return self._file.read(length)

## BIG Files

In [ ]:
with open(FILE_NAME, "rb") as big:
	stream = FileStream(big)
	stream.seek(-4, 2)

	print("Pos:", hex(stream.tell()))
	size = stream.read_uint32()
	print("Size:", size, hex(size))
	print("Pos:", hex(stream.tell()))

	stream.seek(-size, 2)
	stream.read_pad(0x4)

	print("Pos:", hex(stream.tell()))
	num_files = stream.read_uint32()
	print("Files:", num_files, hex(num_files))
	print("Pos:", hex(stream.tell()))
	print()

	file_data = [None] * num_files

	print(num_files)
	for i in range(num_files):
		hash = stream.read_uint32()
		offset = stream.read_uint32()
		size = stream.read_uint32()
		stream.read_pad(4)
		xsize = stream.read_uint32()
		offset *= 0x10
		file_data[i] = [hash, offset, size, xsize]

	for i in range(num_files):
		pass

	# stream.seek(-4, 2)
	# print("Pos:", hex(stream.tell()))
	# table_offset = unpack("<I", big.read(4))[0]
	# file_offset = table_offset - 0xC
	# size_offset = table_offset - 0x10
	# print("Size:", size_offset, hex(size_offset))
	# print("Pos:", hex(stream.tell()))

	# big.seek(-size_offset, 2)

	# print("Pos:", hex(stream.tell()))
	# size = unpack("<I", big.read(4))[0]
	# print("Files:", size, hex(num_files))
	# print("Pos:", hex(stream.tell()))

	# print(size)
	# big.seek(4, 1)
	# raw_size = unpack("<I", big.read(4))[0]
	# big.seek(-file_offset, 2)
	# offset = unpack("<I", big.read(4))[0] * 0x10

	# print(size, raw_size, offset)

	# big.seek(offset)

## WAD Files

In [ ]:
with open(FILE_NAME, "rb") as wad:
	stream: FileStream = FileStream(wad)

	sign = stream.read_string(3)
	stream.read_pad(1)
	num_files = stream.read_uint32()
	
	file_data = [None] * num_files
	names_offset = 0

	for i in range(num_files):
		hash = stream.read_uint32()
		offset = stream.read_uint32()
		size = stream.read_uint32()
		names_offset = max(offset + size, names_offset)
		file_data[i] = [hash, offset, size]

		print(hex(offset))

	stream.seek(names_offset, 0)
	for i in range(num_files):
		name_len = stream.read_uint16()
		name = stream.read_string(name_len).decode("utf-8")
		file_data[i].append(name_len)
		file_data[i].append(name)
		offset = file_data[i][1]
		size = file_data[i][2]

		# position = stream.tell()

		# stream.seek(offset, 0)
		# makedirs(join(OUT_DIRECTORY, dirname(name)), exist_ok = True)
		# with open(join(OUT_DIRECTORY, name), "wb") as data_file:
		# 	data_file.write(stream.read_chunk(size))

		# stream.seek(position, 0)


# with open(join(OUT_DIRECTORY, FILE_NAME.split(".")[0], OUT_FILE), "w") as file:
# 	dump(file_data, file, indent = 4)

In [ ]:
DIRECTORY = "X:\\SteamLibrary\\steamapps\\common\\L.A.Noire"

from os import walk
from os.path import join

for root, dirs, files in walk(DIRECTORY, ):
	if "Mods" in root:
		continue
	for file in files:
		with open("out.xml", "a") as f:
			f.write(f"\t<GameFile Name=\"{join(root, file).lstrip(DIRECTORY)}\" />\n")
